# Interpretable AI for Deep Learning Models
###### ARI3205-SEM1-A-2526
### Proffessor: Konstantinos Makantasis
##### Authors: Love Carlander Strandang (2513558), Edvin Sanfridsson (2515058)



# 0.0 Setup

This section contains:
- Imports + environment info
- Reproducibility (seeds)
- Generic training loops (tabular + vision)
- Metrics + plotting utilities
- Model definitions (Tabular NN + CNN)
- Small wrappers to make PyTorch models compatible with interpretability libraries

## 0.1 Setup and Imports
### 0.1.1 Imports

In [19]:
import json
import pickle
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, accuracy_score, mean_absolute_error


### 0.1.2 Reproducibility 
We set a seed for random so that we will get cosistent reproducable results

In [2]:
RANDOM_SEED = 1337
np.random.seed(RANDOM_SEED)

### 0.1.3 Setup Root Repo

In [20]:
current_working_directory = Path.cwd()
repo_root_directory = current_working_directory.parent

if str(repo_root_directory) not in sys.path:
    sys.path.insert(0, str(repo_root_directory))

print("Repo root added to sys.path:", repo_root_directory)

artifacts_root_directory = repo_root_directory / "artifacts"

adult_artifacts_directory = artifacts_root_directory / "train_all" / "adult_mlp"
california_artifacts_directory = artifacts_root_directory / "train_all" / "california_mlp"
fashion_artifacts_directory = artifacts_root_directory / "train_all" / "fashion_cnn"

print("CWD:", current_working_directory)
print("Repo root:", repo_root_directory)
print("Artifacts root exists?", artifacts_root_directory.exists())
print("Adult artifacts exists?", adult_artifacts_directory.exists())
print("California artifacts exists?", california_artifacts_directory.exists())
print("Fashion artifacts exists?", fashion_artifacts_directory.exists())



Repo root added to sys.path: /Users/nordh/Documents/Software-Engineering-and-Managment-GU/MALTA/Courses/Interperatable AI/Interperatable
CWD: /Users/nordh/Documents/Software-Engineering-and-Managment-GU/MALTA/Courses/Interperatable AI/Interperatable/notebooks
Repo root: /Users/nordh/Documents/Software-Engineering-and-Managment-GU/MALTA/Courses/Interperatable AI/Interperatable
Artifacts root exists? True
Adult artifacts exists? True
California artifacts exists? True
Fashion artifacts exists? True


## 0.2 Load Trained Models

### 0.2.1 Adult
We have trained the

In [12]:
import tensorflow as tf
from tensorflow import keras

adult_model_path = adult_artifacts_directory / "best_model.keras"
adult_preprocessor_path = adult_artifacts_directory / "preprocessing.pkl"
adult_feature_names_path = adult_artifacts_directory / "expanded_feature_names.json"

adult_model = keras.models.load_model(adult_model_path)

with open(adult_preprocessor_path, "rb") as file_handle:
    adult_preprocessor = pickle.load(file_handle)

adult_expanded_feature_names = json.loads(adult_feature_names_path.read_text())

#### Prediction Wrappers

In [13]:
def predict_adult_positive_probability(features_dataframe: pd.DataFrame) -> np.ndarray:
    processed_features = adult_preprocessor.transform(features_dataframe)
    probability = adult_model.predict(processed_features, verbose=0).reshape(-1)
    return probability

def predict_adult_label(features_dataframe: pd.DataFrame, threshold: float = 0.5) -> np.ndarray:
    probability = predict_adult_positive_probability(features_dataframe)
    return (probability >= threshold).astype(int)

### 0.2.2 California (Keras regressor + numeric preprocessor)

In [14]:
california_model_path = california_artifacts_directory / "best_model.keras"
california_preprocessor_path = california_artifacts_directory / "preprocessing.pkl"
california_feature_names_path = california_artifacts_directory / "feature_names.json"

california_model = keras.models.load_model(california_model_path)

with open(california_preprocessor_path, "rb") as file_handle:
    california_preprocessor = pickle.load(file_handle)

california_feature_names = json.loads(california_feature_names_path.read_text())


In [15]:
def predict_california_value(features_dataframe: pd.DataFrame) -> np.ndarray:
    processed_features = california_preprocessor.transform(features_dataframe)
    prediction = california_model.predict(processed_features, verbose=0).reshape(-1)
    return prediction


### 2.3 Fashion CNN (PyTorch)

In [21]:
import torch
from src.models.fashion_cnn import FashionCNN  # your class
from src.data.fashion import load_fashion, FASHION_MNIST_CLASS_NAMES

fashion_weights_path = fashion_artifacts_directory / "best_model.pt"

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

fashion_model = FashionCNN(class_count=len(FASHION_MNIST_CLASS_NAMES)).to(device)
fashion_model.load_state_dict(torch.load(fashion_weights_path, map_location=device))
fashion_model.eval();

fashion_train_dataset, fashion_test_dataset = load_fashion(normalize=False)  # match your training choice

## 0.3 Load test splits

### 0.3.1 Adult splits

In [22]:
from src.data.adult import load_adult_splits

adult_train_df, adult_val_df, adult_test_df, y_train, y_val, y_test, adult_meta = load_adult_splits(
    random_state=RANDOM_SEED
)

### 0.3.2 Calefornia Splits

In [23]:
from src.data.california import load_california

cal_train_df, cal_val_df, cal_test_df, cal_y_train, cal_y_val, cal_y_test = load_california(
    random_state=RANDOM_SEED
)

### 0.3.3 Sanity check

In [25]:
adult_test_probs = predict_adult_positive_probability(adult_test_df)
adult_test_pred = (adult_test_probs >= 0.5).astype(int)
print("Adult test accuracy:", accuracy_score(y_test, adult_test_pred))
print("Adult test AUC:", roc_auc_score(y_test, adult_test_probs))

cal_test_pred = predict_california_value(cal_test_df)
print("California test MAE:", mean_absolute_error(cal_y_test, cal_test_pred))

Adult test accuracy: 0.8469648889343843
Adult test AUC: 0.9041485919101298
California test MAE: 0.7705781851208555
